<a href="https://colab.research.google.com/github/dyphen12/quabity-ashuance/blob/main/Data_Quality_sanitized%5Bmodular%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Quality Report

All functions.

- Run first

In [ ]:
import os, yaml

os.environ['http_proxy'] = "http://proxy-dmz.intel.com:911" 
os.environ['https_proxy'] = "http://proxy-dmz.intel.com:912" 
os.environ['no_proxy'] = "intel.com,.intel.com,10.0.0.0/8,192.168.0.0/16,localhost,::1,.local,127.0.0.0/8,134.134.0.0/16,172.16.0.0/12"

#!pip install pyodbc
#!pip install missingno
#!pip install dataframe_image
#!pip install fpdf
#!pip install pdfkit
#!pip install pandas-profiling

import pyodbc
from time import time
from datetime import timedelta
from dateutil.relativedelta import relativedelta
import datetime
import string
import re
import warnings

from collections import Counter
 
import pandas as pd
import numpy as np
from numpy import unravel_index
import missingno as msno
from pandas_profiling import ProfileReport

import dataframe_image as dfi
from fpdf import FPDF
import pdfkit

In [ ]:
def load_data():
    sourcedata1 = pd.read_csv('sourcedata1.csv')
    sourcedata2 = pd.read_csv('sourcedata2.csv')
    return sourcedata1, sourcedata2

In [ ]:
def load_data_multiple():
    import os
    arr = os.listdir('files')
    
    datalst = []
    
    id = 'Process ID'
    
    
    
    
    for filename in arr:
        flname = 'files/' + filename
        df = pd.read_csv(flname)
        df[id] = df[id].astype(float)
        datalst.append(df)
        print(filename)
    return datalst

In [ ]:
def data_format_multiple(df_list):
    #df1 = sourcedata1
    #df2 = sourcedata2
    id = 'Process ID'
    #df1[id] = df1[id].astype(float)
    #df2[id] = df2[id].astype(float)
    #Merge sourcedata
    ##################Request: for all dfs available, not just 1 or 2
    # del data
    #data = df1.merge(df2, how='left', on='Process ID')
    data = pd.concat(df_list)
    #Show available data columns
    data.columns
    #Select columns for data quality analysis (in main file)
    cols = ['Process ID', 'Process Name', 'Process Purpose', 'Product Line', 'Organization Name',
        'HRGO Process', 'Execution Frequency', 'Automation Level',
        'Financial Impact','Employees Served', 'Executives Served', 'Region', 'Country', 'DFE EE Population', 'DFE VP Population', 'Modified', 'Created']
    #Final data for profiling
    data = data[cols]
    return data

In [ ]:
def data_format(df1, df2):
    #df1 = sourcedata1
    #df2 = sourcedata2
    id = 'Process ID'
    df1[id] = df1[id].astype(float)
    df2[id] = df2[id].astype(float)
    #Merge sourcedata
    ##################Request: for all dfs available, not just 1 or 2
    # del data
    data = df1.merge(df2, how='left', on='Process ID')
    #Show available data columns
    data.columns
    #Select columns for data quality analysis (in main file)
    cols = ['Process ID', 'Process Name', 'Process Purpose', 'Product Line', 'Organization Name',
        'HRGO Process', 'Execution Frequency', 'Automation Level',
        'Financial Impact','Employees Served', 'Executives Served', 'Region', 'Country', 'DFE EE Population', 'DFE VP Population', 'Modified', 'Created']
    #Final data for profiling
    data = data[cols]
    return data

In [ ]:
def clean_data(data):
    # Replace other kinds of notations and blank fields for null values (nan):
    null_values = ['NA', '?', 'Nil','None', '', 'NULL']
    #Replace all the null values with na
    data = data.replace(null_values, np.nan)
    return data

In [ ]:
def profiling_by_dimension(data):
    #Completeness
    #General completeness of data
    completeness = (data.notna().sum().sum())/(data.count().sum())*100

    #Completeness per column
    data_completeness = pd.DataFrame(
        data.notna().sum(),
        columns = ['Completeness']
    )
    #Validity
    #Validity per column
    data_type = pd.DataFrame(
        data.dtypes,
        columns = ['Data Format']
    )
    
    #############Request:To check validity, we need to assign an ideal data type for each column in data. can we do this in a simpler way?
    types = [
    #Process ID
    'float64',
    #Process Name
    'object',
    #Process Purpose
    'object',
    #Product Line
    'object',
    #Organization Name
    'object',
    #HRGO Process
    'object',
    #Execution Frequency
    'object',
    #Automation Level
    'object',
    #Financial Impact
    'float64',
    #Employees Served
    'int64',
    #Executives Served
    'int64',
    #Region
    'object',
    #Country
    'object',    
    #DFE EE Population
    'float64', 
    #DFE VP Population
    'float64',
    #Created
    'datetime64[ns]',
    #Modified
    'datetime64[ns]',
    ]

    data_validity = pd.DataFrame(
        {'Expected Format': types},
        index=data_type.index
    )
    #Validity per column
    data_validity = data_type.join(data_validity)
    data_validity['Validity %'] = np.where((data_validity['Data Format'] == data_validity['Expected Format']), 100, 0)
    #General Validity
    validity = round(data_validity['Validity %'].mean(),2)
    #Nullity-Missing values
    #General nullity for data
    nullity = round((data.isna().sum().sum())/(data.count().sum())*100,2)
    #Nullity per colum
    missing_data = pd.DataFrame(
        data.isnull().sum(), #To see by percentage /data.sum()*100
        columns=['Missing Values']
    )
    #Missing Data Graph
    fig = msno.matrix(data, sparkline=False, figsize=(10,5), fontsize=12, color=(0.27, 0.52, 1.0))
    fig_copy = fig.get_figure()
    fig_copy.savefig('missing.png', bbox_inches = 'tight')
    #Uniqueness
    #General duplicate identification
    duplicates = 100 - (data.duplicated().sum()/len(data)*100)

    #Duplicates by column
    unique_values = pd.DataFrame(
        columns=['Unique Values']
    )
    for row in list(data.columns.values):
        unique_values.loc[row] = [data[row].nunique()]
        
    #Zero Values
    #Zero Values by column
    zero_values = pd.DataFrame(
        data.eq(0).sum(), #Para ver por porcentaje /data.sum()*100
        columns=['Zero Values']
    )
    
    #Minimum values by column
    minimum_values = pd.DataFrame(
        columns=['Minimum Value']
    )
    for row in list(data.columns.values):
        minimum_values.loc[row] = [data[row].dropna().min()]
        
    #Maximum values by column
    maximum_values = pd.DataFrame(
        columns=['Maximum Value']
    )
    for row in list(data.columns.values):
        maximum_values.loc[row] = [data[row].dropna().max()]
        
    #Timeliness
    #Timeliness is intented as data updated over the past two years from present.  Only available for entire data, not by column.
    from datetime import date
    data['Year'] = pd.DatetimeIndex(data['Modified']).year
    data['Updated'] = np.where((data['Year'] >= date.today().year-2), 1, 0) #We consider updated everything modified in current or past year)

    timeliness = data['Updated'].sum()/len(data)*100
    
    #Create table with Process IDs for checking
    #We select process with high delta between dfe population and employees served as candidates for updating.
    data['EE Delta'] = (data['DFE EE Population']-data['Employees Served'])/data['DFE EE Population']
    data['Check EE Population'] = np.where(((data['EE Delta'] >= 0.5) | (data['EE Delta'] <= -0.5)), 1, 0) #We consider updated everything modified in current or past year)
    check_ee = data['Check EE Population'].sum()

    data['VP Delta'] = (data['DFE VP Population']-data['Executives Served'])/data['DFE VP Population']
    data['Check VP Population'] = np.where(((data['VP Delta'] >= 0.5) | (data['VP Delta'] <= -0.5)), 1, 0) #We consider updated everything modified in current or past year)
    check_vp = data['Check VP Population'].sum()

    #Select Check Popolation
    processes = data.loc[data['Check EE Population'] == 1] 
    
    #Join analysis in table
    #del dq_report
    dq_report = data_completeness.join(data_validity).join(missing_data).join(zero_values).join(unique_values).join(minimum_values).join(maximum_values)
    #Conditional Formating
    kpi = 98

    #Color highlighting
    def highlighter(cell_value):

        highlight = 'background-color: yellow;'
        default = 'background-color: lightgreen;'

        if type(cell_value) in [float, int]:
            if cell_value < kpi:
                return highlight
        return default

    def highlighter_complete(cell_value):

        highlight = 'background-color: yellow;'
        default = 'background-color: lightgreen;'

        if type(cell_value) in [float, int]:
            if cell_value == len(data):
                return default
        return highlight

    def highlighter_missing(cell_value):

        highlight = 'background-color: yellow;'
        default = 'background-color: lightgreen;'

        if type(cell_value) in [float, int]:
            if cell_value == 0:
                return default
        return highlight

    dq_report=(
        dq_report
        .style
        .set_table_styles([dict(props=[('max-width', '1200px')])])
        .applymap(highlighter,  subset=['Validity %'])
        .applymap(highlighter_missing,  subset=['Missing Values', 'Zero Values'])
        .applymap(highlighter_complete, subset=['Completeness'])

    )
    
    #Print table to image
    dfi.export(dq_report, "dq_report.png")
    
    #Create full data report
    full_data = pd.DataFrame(
        {'Completeness %': completeness, 'Validity %': validity, 'Nullity %':nullity,'Uniqueness %':duplicates,
         'Timeliness %':timeliness, 'Check EE Population': check_ee, 'Check VP Population': check_vp}, index=[0]
    )
    full_data = full_data.style.hide_index().format('{:.2f}')
    
    #Conditional formating
    full_data=(
        full_data
        .applymap(highlighter,  subset=['Completeness %', 'Validity %', 'Uniqueness %', 'Timeliness %'])
        .applymap(highlighter_missing,  subset=['Nullity %']))
    
    #Print table to image
    dfi.export(full_data, "full_data.png")
    
    #Create pdf report

    pdf = FPDF()
    pdf.add_page()

    #Add Title
    title = 'Data Quality Profiler'
    pdf.set_title(title)
    # Arial bold 15
    pdf.set_font('Arial', 'B', 14)
    # Calculate width of title and position
    w = pdf.get_string_width(title) + 6
    pdf.set_x((210 - w) / 2)
    # Colors of frame, background and text
    pdf.set_draw_color(255, 255, 255)
    pdf.set_fill_color(255, 255, 255)
    pdf.set_text_color(65,105,225)
    # Thickness of frame (1 mm)
    pdf.set_line_width(1)
    # Add title
    pdf.cell(w, 9, title, 1, 1, 'C', 1)
    # Line break
    pdf.ln(4)

    #Data Table
    pdf.set_font('Arial', 'B', 10)
    pdf.set_text_color(30,144,255)
    pdf.cell(0, 5, 'Quality Profiling', 1, 1, 'L')
    pdf.ln(1)
    pdf.image('full_data.png', w = 120, h = 8)
    pdf.ln(3)

    #Column Table
    pdf.set_font('Arial', 'B', 10)
    pdf.set_text_color(30,144,255)
    pdf.cell(0, 5, 'Quality Profiling by Column', 1, 1, 'L')
    pdf.ln(1)
    pdf.image('dq_report.png', w = 135,h = 150)
    pdf.ln(3)

    #Missing Graph
    pdf.set_font('Arial', 'B', 10)
    pdf.set_text_color(30,144,255)
    pdf.cell(0, 5, 'Missing Data Matrix', 1, 1, 'L')
    pdf.ln(3)
    pdf.image('missing.png', w = 100, h = 50)

    #Dimension explanation
    pdf.add_page()
    text = """
    Top data quality dimensions to assess:
    Completess: filled-out values, even with blanks or zeros.
    Nullity: missing data with null/nan values.
    Uniqueness: number of unique values with no duplicates.
    Validity: number of unique values with the correct format (object, float, datetime, etc).
    Timeliness: information available when it is required. In this instance, it refers to the % of processes updated in the past 2 years.
    Other dimensions for further analysis:
    Accuracy: it describes how well does a piece of information reflect reality. It can be validated by process owners or decision makers in general.
    Consistency: values that match values elsewhere.  For instance, elements of a form or tool.
    """
    pdf.set_font('Arial', '', 7)
    pdf.set_text_color(0,0,0)
    pdf.multi_cell(0, 3, text)

    pdf.output('QualityReport.pdf', 'F')
    
    #Print data to CSV
    data.to_csv('processchecklist.csv')
    
    profile = ProfileReport(data, title="Quality Profiling Report")
    
    return profile

In [ ]:
def full_profile_variable(profile):
    #profile = ProfileReport(data, title="Quality Profiling Report")
    profile.to_file("fulldataqualityreport.html")
    return profile

In [ ]:
def full_investigation_by_column(data):
    #Investigate Unique values for specific columns
    #Indicate column
    column= 'Employees Served'
    data.groupby(column)['Process ID'].nunique()
    #Check for updated data by year
    #Select date field
    date = 'Year'
    data.groupby(date)['Process ID'].nunique()

# Main modules

### Loads only df1 and df2

In [ ]:
s1, s2 = load_data()
rawdata = data_format(s1, s2)
cdata = clean_data(rawdata)
pdata = profiling_by_dimension(cdata)
pfile = full_profile_variable(pdata)
full_investigation_by_column(cdata)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

f:\users\möebius\documents\prisma venv\aivenv\lib\site-packages\numpy\core\_methods.py:202: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### Load all datasets allocated at 'files' folder

Instruction: You need to create the 'files' folder at root and save all the files there

In [ ]:
datas = load_data_multiple()
rawdata = data_format_multiple(datas)
cdata = clean_data(rawdata)
pdata = profiling_by_dimension(cdata)
pfile = full_profile_variable(pdata)
full_investigation_by_column(cdata)

sourcedata1.csv
sourcedata2.csv
sourcedata3.csv
sourcedata4.csv
sourcedata5.csv
sourcedata6.csv


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

f:\users\möebius\documents\prisma venv\aivenv\lib\site-packages\pandas_profiling\model\correlations.py:61: UserWarning: There was an attempt to calculate the cramers correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"cramers": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/pandas-profiling/pandas-profiling/issues
(include the error message: 'No data; `observed` has size 0.')
  (include the error message: '{error}')"""


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]